In [16]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

# if os.getlogin()=="JVARGH7":
#    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
#   path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

# elif os.getlogin()=='aamnasoniwala':
#    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
#    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "\llm training\Test Data Splits\Test Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")

<>:25: SyntaxWarning: invalid escape sequence '\l'
<>:25: SyntaxWarning: invalid escape sequence '\l'
/var/folders/k7/_r7nqc9s78jfx1ftfzjxmfwc0000gn/T/ipykernel_48744/3783868891.py:25: SyntaxWarning: invalid escape sequence '\l'
  excel_path = path_equity_precision_llm_folder + "\llm training\Test Data Splits\Test Part "


In [ ]:
excel_path_test = path_equity_precision_llm_folder + "/llm training/epldat03_Test Data.xlsx"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")

ValueError: Unsupported file format. Please provide an Excel or CSV file.

In [ ]:
input_test = clean_input(input_path=excel_path_test, sheet_name='Sheet1')

ValueError: Unsupported file format. Please provide an Excel or CSV file.

In [ ]:
import pandas as pd
import re

n_json_splits = 4
results = None

# Concatenate multiple CSVs
for scenario in range(1, n_json_splits + 1):
    file_path = f"{path_equity_precision_llm_folder}/llm training/Test Data Splits/Test Part {scenario}_results.csv"
    temp_df = pd.read_csv(file_path)

    if results is None:
        results = temp_df
    else:
        results = pd.concat([results, temp_df], ignore_index=True)

# Merge datasets
merged_df_test = input_test.merge(results, left_on='PMID', right_on='pmid', how='left')

# Standardize source population categories
merged_df_test['orig_source_population'] = merged_df_test['orig_source_population'].apply(standardize_population)
merged_df_test['gpt_source_population'] = merged_df_test['gpt_source_population'].apply(standardize_population)

# Classification comparison
merged_df_test['source_population_match'] = merged_df_test.apply(adjusted_source_population_match, axis=1)

# Crosstab summaries for individual conditions
summary_precision_medicine = crosstab_summary(merged_df_test, truth='orig_precision_medicine', test='gpt_precision_medicine')
summary_diabetes = crosstab_summary(merged_df_test, truth='orig_diabetes', test='gpt_diabetes')
summary_primary_study = crosstab_summary(merged_df_test, truth='orig_primary_study', test='gpt_primary_study')

summary_precision_medicine['variable'] = 'Precision Medicine'
summary_diabetes['variable'] = 'Diabetes'
summary_primary_study['variable'] = 'Primary Study'

# Source population accuracy
t_source_population = pd.crosstab(merged_df_test['source_population_match'], merged_df_test['orig_source_population'])
prop_correct_source_population = t_source_population.loc[True].sum() / t_source_population.sum().sum()
summary_source_population = pd.DataFrame({'variable': ['Source Population'],
                                          'Accuracy': [prop_correct_source_population]})

# Overall total accuracy
merged_df_test['all_conditions_correct'] = (
    (merged_df_test['orig_precision_medicine'] == merged_df_test['gpt_precision_medicine']) &
    (merged_df_test['orig_diabetes'] == merged_df_test['gpt_diabetes']) &
    (merged_df_test['orig_primary_study'] == merged_df_test['gpt_primary_study'])
)

overall_accuracy = merged_df_test['all_conditions_correct'].mean()
summary_overall_conditions = pd.DataFrame({
    'variable': ['Overall Conditions (Precision Medicine, Diabetes, Primary Study)'],
    'Accuracy': [overall_accuracy]
})

# Combine all summaries into one df
df_summary = pd.concat([
    summary_precision_medicine,
    summary_diabetes,
    summary_primary_study,
    summary_source_population,
    summary_overall_conditions
])

# Save combined output
df_summary.to_csv(f"{path_equity_precision_llm_repo}/preprocessing/epl04_combined output_Test.csv", index=False)

NameError: name 'input_test' is not defined